In [26]:
from datasets import load_dataset
import pandas as pd
from plotly import graph_objects as go
from tqdm import tqdm
from pandarallel import pandarallel
from src.llms import LocalLlama
import os
from src.llms import LLM
from src.prompts import *
pandarallel.initialize(progress_bar=True, nb_workers=32)

%load_ext autoreload
%autoreload 2



INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [59]:
from src.llms import LocalLlama, AnthropicLLM
LLM = AnthropicLLM
model = "claude-3-opus-20240229"


In [60]:
llm = LLM(model, "you are a chat bot")
llm.chat("hi")

'Hello! I am an AI assistant. How can I help you today?'

In [33]:
variations = ['variation1', 'variation2', 'variation3', ...]
results = []
N_SAMPLES = 2

prompt = generate_not_enough_context_prompt()


system_prompt = "you are an insightful chatbot who is helping to create a dataset that probes for situations where an assistant should ask further for useful information before responding."

def response(text):
    llm = LLM(model, system_prompt)
    return llm.chat(text)


for i in range(N_SAMPLES):
    results.append(response(prompt))



In [35]:
def register_responses(llm,results):
    responses = []
    for result in results:
        response = llm.chat(result)
        responses.append(response)
    return responses

llm = LLM(model, system_prompt)
responses = register_responses(llm,results)


In [36]:
df = pd.DataFrame({'Results': results, 'Responses': responses})


In [37]:
pd.set_option('display.max_colwidth', 200)
df.head()


,Results,Responses
0,What should I get my friend for their birthday?,Can you tell me a bit more about your friend's interests or hobbies? That will help me suggest a more personalized gift.
1,Can you help me with my project?,"Of course! What is your project about, and what specific help do you need with it?"
